In [1]:
!pip install spektral
!pip install Keras
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 4.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import os
import networkx as nx
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

from spektral.layers import GCNConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

from collections import Counter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [6]:
!unzip "cora (extract.me).zip"

Archive:  cora (extract.me).zip
  inflating: cora/README             
  inflating: cora/cora.cites         
  inflating: cora/cora.content       


In [7]:
all_data = []
all_edges = []

for root,dirs,files in os.walk('./cora'):
    for file in files:
        if '.content' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines())
        elif 'cites' in file:
            with open(os.path.join(root,file),'r') as f:
                all_edges.extend(f.read().splitlines())

                

all_data = shuffle(all_data,random_state=42)

In [8]:
labels = []
nodes = []
X = []

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(elements[-1])
    X.append(elements[1:-1])
    nodes.append(elements[0])

X = np.array(X,dtype=int)
N = X.shape[0] 
F = X.shape[1]
print('X shape: ', X.shape)


#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))

print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)

X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {'Reinforcement_Learning', 'Genetic_Algorithms', 'Rule_Learning', 'Probabilistic_Methods', 'Theory', 'Neural_Networks', 'Case_Based'}

Number of classes:  7


In [9]:
#get 20 train instances per class
#remaining instances are in test set test  
def limit_data(labels,limit=20,val_num=500,test_num=1000):
    '''
    Get the index of train, validation, and test data
    '''
    label_counter = dict((l, 0) for l in labels)
    train_idx = []

    for i in range(len(labels)):
        label = labels[i]
        if label_counter[label]<limit:
            #add the example to the training data
            train_idx.append(i)
            label_counter[label]+=1
        
        #exit the loop once we found 20 examples for each class
        if all(count == limit for count in label_counter.values()):
            break
    
    #get the indices that do not go to traning data
    rest_idx = [x for x in range(len(labels)) if x not in train_idx]
    val_idx = rest_idx[:val_num]
    test_idx = rest_idx[val_num:(val_num+test_num)]
    return train_idx, val_idx,test_idx

train_idx,val_idx,test_idx = limit_data(labels)

#set the mask
train_mask = np.zeros((N,),dtype=bool)
train_mask[train_idx] = True

val_mask = np.zeros((N,),dtype=bool)
val_mask[val_idx] = True

test_mask = np.zeros((N,),dtype=bool)
test_mask[test_idx] = True

In [10]:
#build the graph
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)

#obtain the adjacency matrix (A)
A = nx.adjacency_matrix(G)


In [24]:


# Parameters
#channels, dropout_prob, learning rate, epochs, etc         

channels = 9    
dropout = 0.6   
learning_rate = 1e-3        
epochs = 144        
         


# Model definition

X_in = Input(shape=(F, ))
fltr_in = Input((N, ), sparse=True)

dropout1 = Dropout(dropout)(X_in)
conv1 = GCNConv(channels,activation='relu',use_bias=False)([dropout1, fltr_in])

dropout2 = Dropout(dropout)(conv1)
conv2 = GCNConv(channels,activation='relu',use_bias=False)([dropout2, fltr_in])

dropout3 = Dropout(dropout)(conv2)
conv3 = GCNConv(num_classes,activation='softmax',use_bias=False)([dropout3, fltr_in])



#DEFINE YOUR MODEL HERE

# Build model
#model = Model(inputs ..., outputs=...)
#optimizer = # your optimizer 
#model.compile(...)
#model.summary()


model = Model(inputs=[X_in, fltr_in], outputs=conv3)

model.compile(optimizer=Adam(lr=learning_rate),loss='categorical_crossentropy',weighted_metrics=['acc'])
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1433)]       0           []                               
                                                                                                  
 dropout_2 (Dropout)            (None, 1433)         0           ['input_3[0][0]']                
                                                                                                  
 input_4 (InputLayer)           [(None, 2708)]       0           []                               
                                                                                                  
 gcn_conv_2 (GCNConv)           (None, 9)            12897       ['dropout_2[0][0]',              
                                                                  'input_4[0][0]']          

In [25]:
# Train model

A = GCNConv.preprocess(A).astype('f4')

labelencoder = LabelEncoder()
encoded = to_categorical(labelencoder.fit_transform(labels))

validation_data = ([X, A], encoded, val_mask)
model.fit([X, A],encoded,sample_weight=train_mask,validation_data=validation_data,epochs=epochs,batch_size=N,shuffle=False)


Epoch 1/144
1/1 [==============================] - 6s 6s/step - loss: 0.1024 - acc: 0.1000 - val_loss: 0.3592 - val_acc: 0.1260
Epoch 2/144
1/1 [==============================] - 0s 151ms/step - loss: 0.1009 - acc: 0.1714 - val_loss: 0.3591 - val_acc: 0.1360
Epoch 3/144
1/1 [==============================] - 0s 146ms/step - loss: 0.1009 - acc: 0.1286 - val_loss: 0.3590 - val_acc: 0.1420
Epoch 4/144
1/1 [==============================] - 0s 170ms/step - loss: 0.1007 - acc: 0.1500 - val_loss: 0.3588 - val_acc: 0.1480
Epoch 5/144
1/1 [==============================] - 0s 169ms/step - loss: 0.1007 - acc: 0.1429 - val_loss: 0.3587 - val_acc: 0.1460
Epoch 6/144
1/1 [==============================] - 0s 157ms/step - loss: 0.1009 - acc: 0.1857 - val_loss: 0.3586 - val_acc: 0.1540
Epoch 7/144
1/1 [==============================] - 0s 176ms/step - loss: 0.1004 - acc: 0.2000 - val_loss: 0.3584 - val_acc: 0.1540
Epoch 8/144
1/1 [==============================] - 0s 157ms/step - loss: 0.1011 - acc:

In [28]:

# Evaluate model


y_pred = model.predict([X[test_mask], A[test_mask,:][:,test_mask]], batch_size=N)
report = classification_report(np.argmax(encoded[test_mask],axis=1), np.argmax(y_pred,axis=1), target_names=labelencoder.classes_)
print('GCN Classification Report: \n {}'.format(report))

1/1 [==============================] - 0s 109ms/step
GCN Classification Report: 
                         precision    recall  f1-score   support

            Case_Based       0.86      0.50      0.64       113
    Genetic_Algorithms       0.93      0.33      0.49       150
       Neural_Networks       0.63      0.45      0.52       306
 Probabilistic_Methods       0.85      0.48      0.61       167
Reinforcement_Learning       0.40      0.89      0.55        79
         Rule_Learning       0.26      0.73      0.39        62
                Theory       0.42      0.76      0.54       123

              accuracy                           0.53      1000
             macro avg       0.62      0.59      0.53      1000
          weighted avg       0.67      0.53      0.54      1000



In [ ]:


# layer_outputs = [layer.output for layer in model.layers]
# activation_model = Model(inputs=model.input, outputs=layer_outputs)
# activations = activation_model.predict([X,A],batch_size=N)


# x_tsne = TSNE(n_components=2).fit_transform(activations[3]) 

# def plot_tSNE(encoded,x_tsne):
#     color_map = np.argmax(encoded, axis=1)
#     plt.figure(figsize=(10,10))
#     for cl in range(num_classes):
#         indices = np.where(color_map==cl)
#         indices = indices[0]
#         plt.scatter(x_tsne[indices,0], x_tsne[indices, 1], label=cl)
#     plt.legend()
#     plt.show()
    
# plot_tSNE(encoded,x_tsne)